# Importing the Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Locating to Drive Location

In [2]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


# Present Working Directory

In [3]:
!pwd

/content/drive/MyDrive


# Installing Hugging face datasets and evaluate Library

In [4]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

# Importing all the relevant packages

In [5]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import (
    Data2VecTextModel,
    Data2VecVisionModel,
    Data2VecAudioModel,
    Data2VecTextForSequenceClassification,
    Data2VecVisionForImageClassification,
    Data2VecAudioForSequenceClassification,
    Trainer,
    TrainingArguments,
    AutoFeatureExtractor,
    AutoTokenizer,
    AutoProcessor,
    EvalPrediction
)
import evaluate
from torch import nn
import math
from typing import Dict, List, Optional, Union

In [6]:
model_name = "facebook/data2vec-text-base"
model = Data2VecTextModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/data2vec-text-base were not used when initializing Data2VecTextModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing Data2VecTextModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecTextModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Data2VecTextModel were not initialized from the model checkpoint at facebook/data2vec-text-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

 # Loading SST-2 and MNLI Dataset from GLUE benchmark

In [7]:
dataset1 = load_dataset("glue", "sst2")
dataset2 = load_dataset("glue", "mnli")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

# Loading the new dataset for Fine-Tuning

In [8]:
new_dataset = load_dataset("ag_news")

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

# Defining Class DORACONFIG Configuration for DoRA (Domain Re-parameterization Adapter)

In [9]:
class DoRAConfig:
    def __init__(
        self,
        r: int = 8,
        alpha: float = 0.5,
        modules_to_save: Optional[List[str]] = None,
    ):
        self.r = r
        self.alpha = alpha
        self.modules_to_save = modules_to_save or []


# Implementation of DoRA (Domain Re-parameterization Adapter) for a layer

In [11]:
class DoRALayer(nn.Module):
    def __init__(
        self,
        original_layer: nn.Linear,
        r: int = 8,
        alpha: float = 0.5,
    ):
        super().__init__()
        self.original_layer = original_layer
        self.r = r
        self.alpha = alpha

        self.lora_A = nn.Parameter(torch.zeros(original_layer.in_features, r))
        self.lora_B = nn.Parameter(torch.zeros(r, original_layer.out_features))

        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        original_output = self.original_layer(x)
        adaptation = x @ self.lora_A @ self.lora_B
        return self.alpha * original_output + (1 - self.alpha) * adaptation


# Method to Apply DoRA to a model by replacing linear layers with DoRA layers

In [13]:
def get_attr_by_name(obj, name):
    for attr in name.split("."):
        obj = getattr(obj, attr)
    return obj

def apply_dora_to_model(model, config: DoRAConfig):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            if not any(key in name for key in config.modules_to_save):
                parent_name, child_name = name.rsplit(".", 1) if "." in name else ("", name)
                parent = model if parent_name == "" else get_attr_by_name(model, parent_name)
                setattr(parent, child_name, DoRALayer(
                    original_layer=module,
                    r=config.r,
                    alpha=config.alpha
                ))
    return model

# Method to Fine-tune the model on a new dataset using DoRA

In [17]:
def fine_tune_with_dora(model, dataset, tokenizer=None, processor=None):
    if "train" in dataset and "test" not in dataset:
        test_dataset = dataset["train"].select(range(100))
        train_dataset = dataset["train"].select(range(100, 1100))
    elif "train" in dataset and "test" in dataset:
        train_dataset = dataset["train"].select(range(min(1000, len(dataset["train"]))))
        test_dataset = dataset["test"].select(range(min(100, len(dataset["test"]))))
    else:
        train_data, test_data = train_test_split(dataset, test_size=0.1, random_state=42)
        train_dataset = train_data
        test_dataset = test_data

    num_labels = len(set(train_dataset["label"]))
    classifier = Data2VecTextForSequenceClassification.from_pretrained(
        "facebook/data2vec-text-base",
        num_labels=num_labels
    )

    dora_config = DoRAConfig(r=8, alpha=0.5, modules_to_save=["classifier"])
    classifier = apply_dora_to_model(classifier, dora_config)

    def preprocess_function(examples):
        return tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=128
        )

    tokenized_train = train_dataset.map(preprocess_function, batched=True)
    tokenized_test = test_dataset.map(preprocess_function, batched=True)

    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=5e-5,
        num_train_epochs=3,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return {
            "accuracy": accuracy_score(labels, predictions),
            "f1": f1_score(labels, predictions, average="weighted")
        }

    trainer = Trainer(
        model=classifier,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    results = trainer.evaluate()

    return classifier, results

# Running the Fine Tunning Loop

In [16]:
fine_tuned_model, ft_results = fine_tune_with_dora(
    model,
    new_dataset,
    tokenizer=tokenizer,
    processor=None
)

Some weights of Data2VecTextForSequenceClassification were not initialized from the model checkpoint at facebook/data2vec-text-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: p24cs0006 (p24cs0006-iit-jodhpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.134223,0.500000,0.388976
2,No log,0.522664,0.870000,0.865811
3,No log,0.430701,0.870000,0.865066


In [17]:
print("Fine-tuning with DoRA")
fine_tuned_model, ft_results = fine_tune_with_dora(
    model,
    new_dataset,
    tokenizer=tokenizer,
    processor=None
)
print(f"Fine-tuning results on new dataset: {ft_results}")

Fine-tuning with DoRA


Some weights of Data2VecTextForSequenceClassification were not initialized from the model checkpoint at facebook/data2vec-text-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.800879,0.790000,0.748226
2,No log,0.468246,0.890000,0.885078
3,No log,0.404230,0.900000,0.894985


Fine-tuning results on new dataset: {'eval_loss': 0.4042300879955292, 'eval_accuracy': 0.9, 'eval_f1': 0.8949854070822605, 'eval_runtime': 0.9256, 'eval_samples_per_second': 108.036, 'eval_steps_per_second': 14.045, 'epoch': 3.0}
